# Notebook for replicating the analyses in "How does a Move to a Flat Tax Affect Montana Household Filers?"
### Richard W. Evans and Paul Johnston, April 2024
This notebook replicates the analyses in the *Research in Focus* article by [Richard W. Evans](https://sites.google.com/site/rickecon) and Paul Johnston entitled "How does a Move to a Flat Tax Affect Montana Household Filers?".

This Jupyter notebook is meant for execution on your local machine and is located at https://github.com/TheCGO/MT-FlatTax/blob/main/MT_FlatTax.ipynb.

## 0. Setting up the environment
To set up the environment:
1. Update your local clone of the `fiscalsim-us` repository on your machine
    * If you don't have the `fiscalsim-us` repository cloned on your machine:
        * Create a fork of the FiscalSim-US main repository (https://github.com/TheCGO/fiscalsim-us) to your GitHub account
        * Navigate to the directory in your terminal where you want to save this repository
        * Clone that fork onto your local machine: `git clone https://github.com/[YourGitHubHandle]/fiscalsim-us.git`
        * Change directory to the new `fiscalsim-us` directory: `cd fiscalsim-us`
        * Create an `upstream` remote that points to the main repository: `git remote add upstream https://github.com/thecgo/fiscalsim-us.git`
    * If you already have the `fiscalsim-us` repository cloned on your machine:
        * Navigate to your `fiscalsim-us` repository directory in your terminal.
        * Make sure you have an `upstream` remote that points to the main `fiscalsim-us` directory
        * Make sure your main branch is updated with the most current changes in the fiscalsim-us `main` directory: `git fetch upstream`
        * Merge those changes into your `main` directory: `git merge upstream/main`
        * Push those changes out to your remote fork: `git push origin main`
2. Create a new branch called `rickecon-pe06220` that comes from Rick's `pe06220` branch
    * Create a new branch called `rickecon-pe06220`: `git checkout -b rickecon-pe06220 main`
    * Pull the changes from Rick's branch into this new branch of your local fork: `git pull https://github.com/rickecon/fiscalsim-us.git pe06220`
3. Create a new conda environment from this `rickecon-pe06220` branch called `fiscalsim-us-dev2`
    * Navigate to the `fiscalsim-us` repository directory in your terminal on your local machine
    * Create the base of the conda environment using the `environment.yml` file in this repository: `conda env create -f environment.yml`
    * Activate the new conda environment: `conda activate fiscalsim-us-dev2`
    * Install fiscalsim-us package directly from this repository's `setup.py` file.
        * For Linux: `pip install -e .[dev]`
        * For Mac: `pip install -e ."[dev]"`
        * For Windows: `pip install -e .'[dev]'`

In your new branch `rickecon-pe06220`, with the new conda environment `fiscalsim-us-dev2` activated, you should be able to run all the analyses below.

### 1. Introduction
Before opening and running this notebook, make sure that you have downloaded or cloned the KS-FlatTax repository (https://github.com/TheCGO/KS-FlatTax) and created and activated the associated conda environment ks-flattax-dev in the environment.yml file.

The notebook for the introduction produces the following figures.

* 1.1. Figure 1. Plot of US states by state employment income tax type

In [ ]:
# Import packages (this might take a little more than a minute)
import pandas as pd
import numpy as np
import geopandas as gpd
from bokeh.io import output_file, output_notebook, export_png
from bokeh.plotting import figure, show
from bokeh.models import (ColumnDataSource, Title, Label, LabelSet, Legend,
                          LegendItem, CategoricalColorMapper, ColorBar,
                          HoverTool, NumeralTickFormatter, GeoJSONDataSource,
                          FactorRange)
from bokeh.models.tickers import SingleIntervalTicker
from bokeh.transform import factor_cmap
from bokeh.sampledata.us_states import data as states
from bokeh.palettes import Category10

import random
import json

from fiscalsim_us.model_api import *
from fiscalsim_us import Simulation
from policyengine_core.reforms import Reform
from policyengine_core.periods import instant

### 1.1. Figure 1. Plot of US states by state employment income tax type

Create the data for state tax type

In [ ]:
# Create a DataFrame of states and their tax types as of January 1, 2024
# 0 = No state income tax
# 1 = Flat state income tax rate
# 2 = Nearly flat state income tax rates
# 3 = Progressive state income tax rates
state_taxtype_list = [
    ["Alabama", "AL", 2],
    ["Alaska", "AK", 0],
    ["Arizona", "AZ", 1],
    ["Arkansas", "AR", 2],
    ["California", "CA", 3],
    ["Colorado", "CO", 1],
    ["Connecticut", "CT", 3],
    ["Delaware", "DE", 3],
    ["District of Columbia", "DC", 3],
    ["Florida", "FL", 0],
    ["Georgia", "GA", 1],
    ["Hawaii", "HI", 3],
    ["Idaho", "ID", 1],
    ["Illinois", "IL", 1],
    ["Indiana", "IN", 1],
    ["Iowa", "IA", 2],
    ["Kansas", "KS", 3],
    ["Kentucky", "KY", 1],
    ["Louisiana", "LA", 3],
    ["Maine", "ME", 3],
    ["Maryland", "MD", 3],
    ["Massachusetts", "MA", 1],
    ["Michigan", "MI", 1],
    ["Minnesota", "MN", 3],
    ["Mississippi", "MS", 1],
    ["Missouri", "MO", 2],
    ["Montana", "MT", 2],
    ["Nebraska", "NE", 3],
    ["Nevada", "NV", 0],
    ["New Hampshire", "NH", 0],
    ["New Jersey", "NJ", 3],
    ["New Mexico", "NM", 3],
    ["New York", "NY", 3],
    ["North Carolina", "NC", 1],
    ["North Dakota", "ND", 3],
    ["Ohio", "OH", 3],
    ["Oklahoma", "OK", 2],
    ["Oregon", "OR", 3],
    ["Pennsylvania", "PA", 1],
    ["Rhode Island", "RI", 3],
    ["South Carolina", "SC", 3],
    ["South Dakota", "SD", 0],
    ["Tennessee", "TN", 0],
    ["Texas", "TX", 0],
    ["Utah", "UT", 1],
    ["Virginia", "VA", 3],
    ["Vermont", "VT", 3],
    ["Washington", "WA", 0],
    ["West Virginia", "WV", 3],
    ["Wisconsin", "WI", 3],
    ["Wyoming", "WY", 0],
]
state_taxtype_df = pd.DataFrame(
    state_taxtype_list, columns=["State", "Abbrev", "TaxType"]
)
state_taxtype_df["TaxType_str"] = ""
state_taxtype_df["TaxType_str"][state_taxtype_df["TaxType"]==0] = \
    "No state labor income tax"
state_taxtype_df["TaxType_str"][state_taxtype_df["TaxType"]==1] = \
    "Flat state income tax rate"
state_taxtype_df["TaxType_str"][state_taxtype_df["TaxType"]==2] = \
    "Nearly flat state income tax rates"
state_taxtype_df["TaxType_str"][state_taxtype_df["TaxType"]==3] = \
    "Progressive state income tax rates"
state_taxtype_df["TaxType_str_short"] = ""
state_taxtype_df["TaxType_str_short"][state_taxtype_df["TaxType"]==0] = \
    "No income tax"
state_taxtype_df["TaxType_str_short"][state_taxtype_df["TaxType"]==1] = \
    "Flat income tax rate"
state_taxtype_df["TaxType_str_short"][state_taxtype_df["TaxType"]==2] = \
    "Nearly flat income tax rates"
state_taxtype_df["TaxType_str_short"][state_taxtype_df["TaxType"]==3] = \
    "Progressive income tax rates"
state_taxtype_df["fill_color"] = ""
state_taxtype_df["fill_color"][state_taxtype_df["TaxType"]==0] = "#D8D3D3"
state_taxtype_df["fill_color"][state_taxtype_df["TaxType"]==1] = "red"
state_taxtype_df["fill_color"][state_taxtype_df["TaxType"]==2] = "purple"
state_taxtype_df["fill_color"][state_taxtype_df["TaxType"]==3] = "blue"

# Sort alphabetically by full state name
state_taxtype_df.sort_values("State", inplace=True, ignore_index=True)

# Create sub-DataFrames for each tax type
no_tax_states_df = \
    state_taxtype_df[state_taxtype_df["TaxType"]==0].reset_index(drop=True)
flat_tax_states_df = \
    state_taxtype_df[state_taxtype_df["TaxType"]==1].reset_index(drop=True)
nflat_tax_states_df = \
    state_taxtype_df[state_taxtype_df["TaxType"]==2].reset_index(drop=True)
prog_tax_states_df = \
    state_taxtype_df[state_taxtype_df["TaxType"]==3].reset_index(drop=True)
print(state_taxtype_df.groupby("TaxType").count())

# Save state_data_df as .csv
state_taxtype_df.to_csv('./data/state_taxtype.csv', index=False)
state_taxtype_df

In [ ]:
# Print the no tax states
no_tax_states_df

In [ ]:
# Print the flat tax states
flat_tax_states_df

In [ ]:
# Print the nearly flat tax states
nflat_tax_states_df

In [ ]:
# Print the progressive tax states
prog_tax_states_df

Create the Bokeh state map figure.

In [ ]:
fig1_title = (
    "Figure 1. Type of state employment income tax system as of January 1, " +
    "2024"
)
# fig1_title = ""
output_file(
    "./images/state_taxtype_2024.html", title=fig1_title, mode='inline'
)
output_notebook()

# Download U.S. states shape files from US Census Bureau
# https://www.census.gov/geographies/mapping-files/2018/geo/carto-boundary-file.html
us_shapefile_path = ("https://github.com/TheCGO/MT-FlatTax/raw/main/data/" +
                     "cb_2018_us_state_20m/cb_2018_us_state_20m.shp")
gdf = gpd.GeoDataFrame.from_file(us_shapefile_path)
gdf_json = gdf.to_json()
gjson = json.loads(gdf_json)

# Remove Puerto Rico from data
del(gjson["features"][7])

# Alaska
# Fix positive longitudes
min_lat_ak = 180  # initial value that will be adjusted
min_abs_lon_ak = 180  # initial value that will be adjusted
coords_list = gjson["features"][24]["geometry"]["coordinates"]
for ind_isl, island in enumerate(coords_list):
    for ind_pnt, point in enumerate(island[0]):
        min_lat_ak = np.minimum(min_lat_ak, point[1])
        if point[0] > 0:
            gjson["features"][24]["geometry"][
                "coordinates"
            ][ind_isl][0][ind_pnt][0] = -180 - (180 - point[0])
        else:
            min_abs_lon_ak = np.minimum(min_abs_lon_ak, -point[0])

# Shrink the size of Alaska relative to its southestern most minimum lattitude
# and longitude
shrink_pct_ak = 0.65
coords_list_ak = gjson["features"][24]["geometry"]["coordinates"]
for ind_isl, island in enumerate(coords_list_ak):
    for ind_pnt, point in enumerate(island[0]):
        gjson["features"][24]["geometry"][
            "coordinates"
        ][ind_isl][0][ind_pnt][0] = point[0] - shrink_pct_ak * (point[0] +
                                                                min_abs_lon_ak)
        gjson["features"][24]["geometry"][
            "coordinates"
        ][ind_isl][0][ind_pnt][1] = point[1] - shrink_pct_ak * (point[1] -
                                                                min_lat_ak)

# Move Alaska closer to the mainland such that the minimum minimum absolute
# longitude and lattitude are (-127, 44)
min_lat_ak_new = 44
min_abs_lon_ak_new = 127
for ind_isl, island in enumerate(coords_list):
    for ind_pnt, point in enumerate(island[0]):
        gjson["features"][24]["geometry"][
            "coordinates"
        ][ind_isl][0][ind_pnt][0] = point[0] + (min_abs_lon_ak -
                                                min_abs_lon_ak_new)
        gjson["features"][24]["geometry"][
            "coordinates"
        ][ind_isl][0][ind_pnt][1] = point[1] - (min_lat_ak - min_lat_ak_new)

# Hawaii
list_ind_hi = 47
# Get minimum lattitude and minimum absolute longitude for Hawaii
min_lat_hi = 180  # initial value that will be adjusted
min_abs_lon_hi = 180  # initial value that will be adjusted
coords_list = gjson["features"][list_ind_hi]["geometry"]["coordinates"]
for ind_isl, island in enumerate(coords_list):
    for ind_pnt, point in enumerate(island[0]):
        min_lat_hi = np.minimum(min_lat_hi, point[1])
        min_abs_lon_hi = np.minimum(min_abs_lon_hi, -point[0])
# print("Minimum lattitude for Hawaii is", min_lat_hi)
# print("Minimum absolute longitude for Hawaii is", min_abs_lon_hi)

# Increase the size of Hawaii
incr_pct_hi = 0.4
coords_list_hi = gjson["features"][list_ind_hi]["geometry"]["coordinates"]
for ind_isl, island in enumerate(coords_list_hi):
    for ind_pnt, point in enumerate(island[0]):
        gjson["features"][list_ind_hi]["geometry"][
            "coordinates"
        ][ind_isl][0][ind_pnt][0] = point[0] + incr_pct_hi * (point[0] +
                                                              min_abs_lon_hi)
        gjson["features"][list_ind_hi]["geometry"][
            "coordinates"
        ][ind_isl][0][ind_pnt][1] = point[1] + incr_pct_hi * (point[1] -
                                                              min_lat_hi)

# Move Hawaii closer to the mainland such that the minimum minimum absolute
# longitude and lattitude are (-125, 27)
min_lat_hi_new = 27.5
min_abs_lon_hi_new = 124.5
for ind_isl, island in enumerate(coords_list):
    for ind_pnt, point in enumerate(island[0]):
        gjson["features"][list_ind_hi]["geometry"][
            "coordinates"
        ][ind_isl][0][ind_pnt][0] = point[0] + (min_abs_lon_hi -
                                                min_abs_lon_hi_new)
        gjson["features"][list_ind_hi]["geometry"][
            "coordinates"
        ][ind_isl][0][ind_pnt][1] = point[1] - (min_lat_hi - min_lat_hi_new)

# Add a state box around Delaware abbreviation DE
st_list_num = 2
de_coord_list = [gjson["features"][st_list_num]["geometry"]["coordinates"]]
new_box_de = [[
    [-75.4, 38.8],
    [-72.4, 38.3],
    [-72.4, 38.9],
    [-70.2, 38.9],
    [-70.2, 37.7],
    [-72.4, 37.7],
    [-72.4, 38.3]
]]
de_coord_list.append(new_box_de)
gjson["features"][st_list_num]["geometry"]["coordinates"] = de_coord_list
gjson["features"][st_list_num]["geometry"]["type"] = "MultiPolygon"

# Add a state box around Washington, DC (District of Columbia) abbreviation DC
st_list_num = 35
dc_coord_list = [gjson["features"][st_list_num]["geometry"]["coordinates"]]
new_box_dc = [[
    [-77.0, 38.9],
    [-73.3, 35.5],
    [-73.3, 36.1],
    [-71.1, 36.1],
    [-71.1, 34.9],
    [-73.3, 34.9],
    [-73.3, 35.5]
]]
dc_coord_list.append(new_box_dc)
gjson["features"][st_list_num]["geometry"]["coordinates"] = dc_coord_list
gjson["features"][st_list_num]["geometry"]["type"] = "MultiPolygon"

# Add a state box around Massachusetts abbreviation MD
st_list_num = 0
md_coord_list = gjson["features"][st_list_num]["geometry"]["coordinates"]
new_box_md = [[
    [-76.8, 39.3],
    [-72.7, 37.0],
    [-72.7, 37.6],
    [-70.5, 37.6],
    [-70.5, 36.4],
    [-72.7, 36.4],
    [-72.7, 37.0]
]]
md_coord_list.append(new_box_md)
gjson["features"][st_list_num]["geometry"]["coordinates"] = md_coord_list

# Add a state box around Massachusetts abbreviation MA
st_list_num = 29
ma_coord_list = gjson["features"][st_list_num]["geometry"]["coordinates"]
new_box_ma = [[
    [-71.7, 42.2],
    [-68.5, 42.2],
    [-68.5, 42.8],
    [-66.3, 42.8],
    [-66.3, 41.6],
    [-68.5, 41.6],
    [-68.5, 42.2]
]]
ma_coord_list.append(new_box_ma)
gjson["features"][st_list_num]["geometry"]["coordinates"] = ma_coord_list

# Add a state box around New Jersey abbreviation NJ
st_list_num = 34
nj_coord_list = [gjson["features"][st_list_num]["geometry"]["coordinates"]]
new_box_nj = [[
    [-74.4, 40.1],
    [-72.0, 39.7],
    [-72.0, 40.3],
    [-69.8, 40.3],
    [-69.8, 39.1],
    [-72.0, 39.1],
    [-72.0, 39.7]
]]
nj_coord_list.append(new_box_nj)
gjson["features"][st_list_num]["geometry"]["coordinates"] = nj_coord_list
gjson["features"][st_list_num]["geometry"]["type"] = "MultiPolygon"

# Add a state box around Rhode Island abbreviation RI
st_list_num = 50
ri_coord_list = gjson["features"][st_list_num]["geometry"]["coordinates"]
new_box_ri = [[
    [-71.5, 41.7],
    [-69.5, 40.4],
    [-69.5, 41.0],
    [-67.3, 41.0],
    [-67.3, 39.8],
    [-69.5, 39.8],
    [-69.5, 40.4]
]]
ri_coord_list.append(new_box_ri)
gjson["features"][st_list_num]["geometry"]["coordinates"] = ri_coord_list

# Merge the state tax type data into gjson for each state
state_taxtype_df
for ind_st, state in enumerate(gjson["features"]):
    st_abbrev = state["properties"]["STUSPS"]
    state["properties"]["tax_type"] = state_taxtype_df[
        state_taxtype_df["Abbrev"]==st_abbrev
    ]["TaxType_str_short"].iloc[0]

tax_type_labels = [
    'No income tax', 'Flat income tax rate', 'Nearly flat income tax rates',
    'Progressive income tax rates'
]
tax_type_colors = ["white", "red", "purple", "blue"]

source_shapes = {}
for category in tax_type_labels:
    source_shapes[category] = {"type": "FeatureCollection", "features": []}

for item in gjson["features"]:
    source_shapes[item["properties"]["tax_type"]]['features'].append(item)

TOOLS = "pan, box_zoom, wheel_zoom, hover, save, reset, help"

fig1 = figure(
    title=fig1_title,
    height=500,
    width=1050,
    tools=TOOLS,
    # tooltips=[
    #     ("State", @state_names), ("Tax type", @tax_type)
    # ]
    # match_aspect = True,
    min_border = 0,
    x_axis_location = None, y_axis_location = None,
    toolbar_location="right"
)
fig1.title.text_font_size = "15pt"
fig1.toolbar.logo = None
fig1.grid.grid_line_color = None

cmap = CategoricalColorMapper(
    palette=tax_type_colors, factors=tax_type_labels
)
for category in tax_type_labels:
    source_shape_1 = GeoJSONDataSource(
        geojson = json.dumps(source_shapes[category])
    )
    fig1.patches(
        'xs', 'ys', source=source_shape_1, fill_alpha=0.7,
        fill_color = {'field': 'tax_type', 'transform': cmap},
        line_color ='black', line_width=1.0, line_alpha=0.3,
        hover_line_color="black", hover_line_width=3.0, legend_label=category)

    hover = fig1.select_one(HoverTool)
    hover.point_policy = "follow_mouse"
    hover.tooltips = [
        ("State", "@NAME"),
        ("Tax type", "@tax_type")
    ]

# Add 2-letter state abbreviation labels. See Bokeh documentation for labels at
# https://docs.bokeh.org/en/latest/docs/user_guide/basic/annotations.html.
label_lon_lat =[
    ["AL",  -87.40, 32.10],
    ["AK", -135.00, 48.30],
    ["AZ", -112.40, 34.00],
    ["AR",  -93.20, 34.30],
    ["CA", -121.00, 37.00],
    ["CO", -106.50, 38.50],
    ["CT",  -73.40, 41.15],
    ["DE",  -72.10, 37.80],
    ["DC",  -73.00, 35.00],
    ["FL",  -82.40, 28.00],
    ["GA",  -84.20, 32.20],
    ["HI", -125.00, 29.00],
    ["ID", -115.00, 43.00],
    ["IL",  -89.60, 39.50],
    ["IN",  -86.90, 39.60],
    ["IA",  -94.40, 41.60],
    ["KS",  -99.50, 38.00],
    ["KY",  -86.00, 37.00],
    ["LA",  -93.00, 30.20],
    ["ME",  -70.00, 44.50],
    ["MD",  -72.40, 36.50],
    ["MA",  -68.20, 41.80],
    ["MI",  -85.40, 42.70],
    ["MN",  -95.30, 45.50],
    ["MS",  -90.60, 32.10],
    ["MO",  -93.50, 38.00],
    ["MT", -110.50, 46.50],
    ["NE", -100.50, 41.00],
    ["NV", -118.00, 39.00],
    ["NH",  -72.45, 42.70],
    ["NM", -107.00, 34.00],
    ["NJ",  -71.60, 39.20],
    ["NY",  -76.00, 42.50],
    ["NC",  -79.20, 35.00],
    ["ND", -101.50, 46.80],
    ["OH",  -83.80, 39.90],
    ["OK",  -98.00, 35.00],
    ["OR", -121.50, 43.50],
    ["PA",  -78.50, 40.40],
    ["RI",  -69.00, 39.90],
    ["SC",  -81.50, 33.30],
    ["SD", -101.00, 44.00],
    ["TN",  -87.10, 35.40],
    ["TX", -100.00, 31.00],
    ["UT", -112.50, 39.00],
    ["VT",  -73.20, 44.10],
    ["VA",  -79.00, 37.10],
    ["WA", -121.00, 47.00],
    ["WV",  -81.90, 38.00],
    ["WI",  -90.40, 44.00],
    ["WY", -108.50, 42.50]
]
label_abbrev = [state[0] for state in label_lon_lat]
label_lon = [state[1] for state in label_lon_lat]
label_lat = [state[2] for state in label_lon_lat]
state_cds = ColumnDataSource(data=dict(
    lon=label_lon,
    lat=label_lat,
    abbrev=label_abbrev
))

state_labels = LabelSet(
    x='lon', y='lat', text='abbrev', text_font_size="9pt",
    text_font_style="bold", x_offset=0, y_offset=0, source=state_cds
)

fig1.add_layout(state_labels)

# Legend properties
fig1.legend.click_policy = 'mute'
fig1.legend.location = "center_left"

fig1.add_layout(
    Title(
        text="  Source: Richard W. Evans (@RickEcon), updated March 30, 2024.",
        align="left",
        text_font_size="3mm",
        text_font_style="italic",
    ),
    "below"
)
show(fig1)

## 2. Montana individual income tax rate landscape
This section contains the code for creating Figures 2 and 3.

### 2.1. Figure 2. Montana current progressive marginal tax rates on employment income versus proposed 5.2% flat tax

In [ ]:
# Create Bokeh plot of Figure 2 current vs proposed flat tax marginal rates
fig2_title = (
    "Fig. 2. Montana current progressive marginal tax rates on employment " +
    "income versus proposed 5.2% flat tax"
)
# fig2_title = ""
filename2 = "./images/CurrReformTaxRates.html"
output_file(filename2, title=fig2_title, mode='inline')
output_notebook()

# Create data objects for plot
flat_tax_rate = 0.052
curr_tax_rates = [0.047, 0.059]
min_marg_tax_rate_toplot = 0.018
max_marg_tax_rate_toplot = 0.065

# Cheap trick to get the y-axis to only show from min_marg_tax_rate_toplot to
# max_margin_tax_rate_toplot
flat_tax_rate_adj = flat_tax_rate - min_marg_tax_rate_toplot
curr_tax_rates_adj = [x - min_marg_tax_rate_toplot for x in curr_tax_rates]
min_marg_tax_rate_toplot_adj = (min_marg_tax_rate_toplot -
                                min_marg_tax_rate_toplot)
max_marg_tax_rate_toplot_adj = (max_marg_tax_rate_toplot -
                                min_marg_tax_rate_toplot)

# Create label for under each bar on the x-axis
bar_under_labels = [
    "$0 to $41k, Married \n $0 to $30,750k for Head of Household \n $0 to $20,500 for other filer types",
    "Greater than $41K, Married \n greater than $30,750 for Head of Household \n greater than $20,500 for other filer types",
]

fig2 = figure(
    height=550,
    width=800,
    x_axis_label="Tax bracket income ranges",
    y_axis_label="Statutory marginal tax rate",
    x_range=bar_under_labels,
    y_range=(min_marg_tax_rate_toplot, max_marg_tax_rate_toplot),
    title=fig2_title,
    tools=["save", "help"],
    toolbar_location="right"
)
fig2.title.text_font_size = "10.5pt"
fig2.toolbar.logo = None

vbar_curr = fig2.vbar(
    x=bar_under_labels, top=curr_tax_rates, color="steelblue", width=0.7,
    border_radius=8
)
fig2.xgrid.grid_line_color = None

# Dashed horizontal line at flat tax 0.052 or 5.2% with fill in shading below
flat_dash = fig2.line(
    x=[0, 3],
    y=[flat_tax_rate, flat_tax_rate],
    color="darkblue",
    line_width=3,
    line_dash="dashed",
    alpha=0.5,
)
fig2.vbar(
    x=1.5, top=flat_tax_rate, color="powderblue", width=3.0, level='underlay'
)

# Add custom y-axis major tick labels
fig2.yaxis.major_label_overrides = {
    0.02: '2%', 0.03: '3%', 0.04: '4%', 0.05: '5%', 0.06: '6%'
}

# Add column height in percent at top center inside of each bar
label_bar1 = Label(
    x=0.45, y=0.042, text='4.7%', x_units='data', y_units='data',
    text_font_size='12pt', text_color='white'
)
fig2.add_layout(label_bar1)
label_bar2 = Label(
    x=1.45, y=0.055, text='5.9%', x_units='data', y_units='data',
    text_font_size='12pt', text_color='white'
)
fig2.add_layout(label_bar2)
label_flat = Label(
    x=0.75, y=0.0522, text='5.2% flat tax rate', x_units='data',
    y_units='data', text_font_size='12pt', text_color='darkblue'
)
fig2.add_layout(label_flat)

# Add legend
legend = Legend(
    items=[
        ("Current policy progressive tax rates", [vbar_curr]),
        ("Proposed flat tax rate", [flat_dash])
    ],
    location="center",
)
fig2.add_layout(legend, "above")
fig2.legend.orientation = "horizontal"
fig2.legend.spacing = 20

# Add note text below figure
fig2.add_layout(
    Title(
        text=(
            "Note: The tax bracket income ranges can be found under each bar."
        ),
        align="left",
        text_font_size="3mm",
        text_font_style="italic",
    ),
    "below",
)

show(fig2)

### 2.1. Figure 3. Montana current standard deduction rate and max versus proposed increases

In [ ]:
# Create Bokeh plot of Figure 3a current vs proposed flat tax standard
# deduction changes to min, max, and rate for filer type Married filing jointly
fig3a_title = (
    "Fig. 3a. Proposed change in MT standard deduction rate, min, and max " +
    "for filer type Married Filing Jointly"
)
# fig3a_title = ""
filename3a = "./images/CurrReformStdDeduc_a.html"
output_file(filename3a, title=fig3a_title, mode='inline')
output_notebook()

# Create data series for plot
rate_base = 0.20
min_base = 4_520
max_base = 10_180
min_agi_max_base = min_base / rate_base
max_agi_min_base = max_base / rate_base

rate_reform = 0.27
min_reform = 5_420
max_reform = 14_180
min_agi_max_reform = min_reform / rate_reform
max_agi_min_reform = max_reform / rate_reform

min_xval = min_agi_max_reform - 4_000
max_xval = max_agi_min_reform + 4_000
min_yval = 2_000
max_yval = 15_500

fig3a = figure(
    height=500,
    width=800,
    x_axis_label="MT adjusted gross income",
    y_axis_label="MT standard deduction",
    y_range=(
        min_yval,
        max_yval
    ),
    x_range=(
        min_xval,
        max_xval
    ),
    tools=[
        "save",
        "hover",
        "help",
    ],
    toolbar_location="right",
)
fig3a.toolbar.logo = None

l_base = fig3a.line(
    x=[min_xval, min_agi_max_base, max_agi_min_base, max_xval],
    y=[min_base, min_base, max_base, max_base],
    color="black",
    line_width=3,
    alpha=0.7,
    muted_alpha=0.15,
)
l_min_agi_max_base = fig3a.line(
    x=[min_agi_max_base, min_agi_max_base],
    y=[min_yval, min_base],
    color="black",
    line_width=2,
    line_dash="dashed",
    alpha=0.7,
    muted_alpha=0.15,
)
l_max_agi_min_base = fig3a.line(
    x=[max_agi_min_base, max_agi_min_base],
    y=[min_yval, max_base],
    color="black",
    line_width=2,
    line_dash="dashed",
    alpha=0.7,
    muted_alpha=0.15,
)
l_base_max_dashed = fig3a.line(
    x=[min_xval, max_agi_min_base],
    y=[max_base, max_base],
    color="black",
    line_width=2,
    line_dash="dashed",
    alpha=0.7,
    muted_alpha=0.15,
)

l_reform = fig3a.line(
    x=[min_xval, min_agi_max_reform, max_agi_min_reform, max_xval],
    y=[min_reform, min_reform, max_reform, max_reform],
    color="red",
    line_width=3,
    alpha=0.7,
    muted_alpha=0.15,
)
l_min_agi_max_reform = fig3a.line(
    x=[min_agi_max_reform, min_agi_max_reform],
    y=[min_yval, min_reform],
    color="red",
    line_width=2,
    line_dash="dashed",
    alpha=0.7,
    muted_alpha=0.15,
)
l_max_agi_min_reform = fig3a.line(
    x=[max_agi_min_reform, max_agi_min_reform],
    y=[min_yval, max_reform],
    color="red",
    line_width=2,
    line_dash="dashed",
    alpha=0.7,
    muted_alpha=0.15,
)
l_base_max_dashed = fig3a.line(
    x=[min_xval, max_agi_min_reform],
    y=[max_reform, max_reform],
    color="red",
    line_width=2,
    line_dash="dashed",
    alpha=0.7,
    muted_alpha=0.15,
)

# Add title
fig3a.add_layout(
    Title(
        text=fig3a_title,
        text_font_style="bold",
        text_font_size="11pt",
        align="center",
    ),
    "above",
)

# Add legend
legend = Legend(
    items=[
        ("Reform: rate=0.27", [l_reform]),
        ("Baseline: rate=0.20", [l_base]),
    ],
    # location="center",
)
fig3a.add_layout(legend)
fig3a.legend.location = "bottom_center"
fig3a.legend.click_policy = "mute"

# Customize the x-axis and y-axis ticks and tick lables and gridlines
fig3a.xaxis.ticker = [
    0, 20_074, 22_600, 30_000, 40_000, 50_900, 52_519
]
fig3a.xaxis.major_label_overrides = {
    0: '$0', 20_074: '$20.1k', 22_600: '$22.6k', 30_000: '$30.0k',
    40_000: '$40.0k', 50_900: '$50.9k', 52_519: '$52.5k'
}
fig3a.yaxis.ticker = [4520, 5420, 10180, 14180]
fig3a.yaxis.major_label_overrides = {
    4520: '$4,520', 5420: '$5,420', 10180: '$10,180', 14180: '$14,180'
}

show(fig3a)

In [ ]:
# Create Bokeh plot of Figure 3b current vs proposed flat tax standard
# deduction changes to min, max, and rate for filer type Head of Household
fig3b_title = (
    "Fig. 3b. Proposed change in MT standard deduction rate, min, and max " +
    "for filer type Head of Household"
)
# fig3b_title = ""
filename3b = "./images/CurrReformStdDeduc_b.html"
output_file(filename3b, title=fig3b_title, mode='inline')
output_notebook()

# Create data series for plot
rate_base = 0.20
min_base = 4_520
max_base = 10_180
min_agi_max_base = min_base / rate_base
max_agi_min_base = max_base / rate_base

rate_reform = 0.27
min_reform = 5_020
max_reform = 14_180
min_agi_max_reform = min_reform / rate_reform
max_agi_min_reform = max_reform / rate_reform

min_xval = min_agi_max_reform - 4_000
max_xval = max_agi_min_reform + 4_000
min_yval = 2_000
max_yval = 15_500

fig3b = figure(
    height=500,
    width=800,
    x_axis_label="MT adjusted gross income",
    y_axis_label="MT standard deduction",
    y_range=(
        min_yval,
        max_yval
    ),
    x_range=(
        min_xval,
        max_xval
    ),
    tools=[
        "save",
        "hover",
        "help",
    ],
    toolbar_location="right",
)
fig3b.toolbar.logo = None

l_base = fig3b.line(
    x=[min_xval, min_agi_max_base, max_agi_min_base, max_xval],
    y=[min_base, min_base, max_base, max_base],
    color="black",
    line_width=3,
    alpha=0.7,
    muted_alpha=0.15,
)
l_min_agi_max_base = fig3b.line(
    x=[min_agi_max_base, min_agi_max_base],
    y=[min_yval, min_base],
    color="black",
    line_width=2,
    line_dash="dashed",
    alpha=0.7,
    muted_alpha=0.15,
)
l_max_agi_min_base = fig3b.line(
    x=[max_agi_min_base, max_agi_min_base],
    y=[min_yval, max_base],
    color="black",
    line_width=2,
    line_dash="dashed",
    alpha=0.7,
    muted_alpha=0.15,
)
l_base_max_dashed = fig3b.line(
    x=[min_xval, max_agi_min_base],
    y=[max_base, max_base],
    color="black",
    line_width=2,
    line_dash="dashed",
    alpha=0.7,
    muted_alpha=0.15,
)

l_reform = fig3b.line(
    x=[min_xval, min_agi_max_reform, max_agi_min_reform, max_xval],
    y=[min_reform, min_reform, max_reform, max_reform],
    color="red",
    line_width=3,
    alpha=0.7,
    muted_alpha=0.15,
)
l_min_agi_max_reform = fig3b.line(
    x=[min_agi_max_reform, min_agi_max_reform],
    y=[min_yval, min_reform],
    color="red",
    line_width=2,
    line_dash="dashed",
    alpha=0.7,
    muted_alpha=0.15,
)
l_max_agi_min_reform = fig3b.line(
    x=[max_agi_min_reform, max_agi_min_reform],
    y=[min_yval, max_reform],
    color="red",
    line_width=2,
    line_dash="dashed",
    alpha=0.7,
    muted_alpha=0.15,
)
l_base_max_dashed = fig3b.line(
    x=[min_xval, max_agi_min_reform],
    y=[max_reform, max_reform],
    color="red",
    line_width=2,
    line_dash="dashed",
    alpha=0.7,
    muted_alpha=0.15,
)

# Add title
fig3b.add_layout(
    Title(
        text=fig3b_title,
        text_font_style="bold",
        text_font_size="11pt",
        align="center",
    ),
    "above",
)

# Add legend
legend = Legend(
    items=[
        ("Reform: rate=0.27", [l_reform]),
        ("Baseline: rate=0.20", [l_base]),
    ],
    # location="center",
)
fig3b.add_layout(legend)
fig3b.legend.location = "bottom_center"
fig3b.legend.click_policy = "mute"

# Customize the x-axis and y-axis ticks and tick lables and gridlines
fig3b.xaxis.ticker = [
    0, 18_593, 22_600, 30_000, 40_000, 50_900, 52_519
]
fig3b.xaxis.major_label_overrides = {
    0: '$0', 18_593: '$18.6k', 22_600: '$22.6k', 30_000: '$30.0k',
    40_000: '$40.0k', 50_900: '$50.9k', 52_519: '$52.5k'
}
fig3b.yaxis.ticker = [4520, 5020, 10180, 14180]
fig3b.yaxis.major_label_overrides = {
    4520: '$4,520', 5020: '$5,020', 10180: '$10,180', 14180: '$14,180'
}

show(fig3b)

In [ ]:
# Create Bokeh plot of Figure 3c current vs proposed flat tax standard
# deduction changes to min, max, and rate for filer types Married filing
# separately, Single, and Widow
fig3c_title = (
    "Fig. 3c. Proposed change in MT standard deduction rate, min, and max " +
    "for filer types Married Filing Separately, Single, and Widow"
)
# fig3c_title = ""
filename3c = "./images/CurrReformStdDeduc_c.html"
output_file(filename3c, title=fig3c_title, mode='inline')
output_notebook()

# Create data series for plot
rate_base = 0.20
min_base = 2_260
max_base = 5_090
min_agi_max_base = min_base / rate_base
max_agi_min_base = max_base / rate_base

rate_reform = 0.27
min_reform = 2_760
max_reform = 7_090
min_agi_max_reform = min_reform / rate_reform
max_agi_min_reform = max_reform / rate_reform

min_xval = min_agi_max_base - 3_000
max_xval = max_agi_min_base + 3_000
min_yval = 1_000
max_yval = 8_500

fig3c = figure(
    height=500,
    width=800,
    x_axis_label="MT adjusted gross income",
    y_axis_label="MT standard deduction",
    y_range=(
        min_yval,
        max_yval
    ),
    x_range=(
        min_xval,
        max_xval
    ),
    tools=[
        "save",
        "hover",
        "help",
    ],
    toolbar_location="right",
)
fig3c.toolbar.logo = None

l_base = fig3c.line(
    x=[min_xval, min_agi_max_base, max_agi_min_base, max_xval],
    y=[min_base, min_base, max_base, max_base],
    color="black",
    line_width=3,
    alpha=0.7,
    muted_alpha=0.15,
)
l_min_agi_max_base = fig3c.line(
    x=[min_agi_max_base, min_agi_max_base],
    y=[min_yval, min_base],
    color="black",
    line_width=2,
    line_dash="dashed",
    alpha=0.7,
    muted_alpha=0.15,
)
l_max_agi_min_base = fig3c.line(
    x=[max_agi_min_base, max_agi_min_base],
    y=[min_yval, max_base],
    color="black",
    line_width=2,
    line_dash="dashed",
    alpha=0.7,
    muted_alpha=0.15,
)
l_base_max_dashed = fig3c.line(
    x=[min_xval, max_agi_min_base],
    y=[max_base, max_base],
    color="black",
    line_width=2,
    line_dash="dashed",
    alpha=0.7,
    muted_alpha=0.15,
)

l_reform = fig3c.line(
    x=[min_xval, min_agi_max_reform, max_agi_min_reform, max_xval],
    y=[min_reform, min_reform, max_reform, max_reform],
    color="red",
    line_width=3,
    alpha=0.7,
    muted_alpha=0.15,
)
l_min_agi_max_reform = fig3c.line(
    x=[min_agi_max_reform, min_agi_max_reform],
    y=[min_yval, min_reform],
    color="red",
    line_width=2,
    line_dash="dashed",
    alpha=0.7,
    muted_alpha=0.15,
)
l_max_agi_min_reform = fig3c.line(
    x=[max_agi_min_reform, max_agi_min_reform],
    y=[min_yval, max_reform],
    color="red",
    line_width=2,
    line_dash="dashed",
    alpha=0.7,
    muted_alpha=0.15,
)
l_base_max_dashed = fig3c.line(
    x=[min_xval, max_agi_min_reform],
    y=[max_reform, max_reform],
    color="red",
    line_width=2,
    line_dash="dashed",
    alpha=0.7,
    muted_alpha=0.15,
)

# Add title
fig3c.add_layout(
    Title(
        text=fig3c_title,
        text_font_style="bold",
        text_font_size="9pt",
        align="center",
    ),
    "above",
)

# Add legend
legend = Legend(
    items=[
        ("Reform: rate=0.27", [l_reform]),
        ("Baseline: rate=0.20", [l_base]),
    ],
    # location="center",
)
fig3c.add_layout(legend)
fig3c.legend.location = "bottom_center"
fig3c.legend.click_policy = "mute"

# Customize the x-axis and y-axis ticks and tick lables and gridlines
fig3c.xaxis.ticker = [
    0, 5_000, 10_222, 11_300, 15_000, 20_000, 25_450, 26_259, 30_000
]
fig3c.xaxis.major_label_overrides = {
    0: '$0', 5_000: '$5k', 10_222: '$10.2k', 11_300: '$11.3k', 15_000: '$15k',
    20_000: '$20k', 25_450: '$25.5k', 26_259: '$26.3k', 30_000: '$30k'
}
fig3c.yaxis.ticker = [2260, 2760, 5090, 7090]
fig3c.yaxis.major_label_overrides = {
    2260: '$2,260', 2760: '$2,760', 5090: '$5,090', 7090: '$7,090'
}

show(fig3c)

## 3. Effects of the flat tax reform on Montana tax filers
### 3.1. Define the 12 filer types
These 12 filer types are detailed in Appendix C of the paper.

#### 3.1.1. Define the three single filer types
Single, no kids, low income (household before-tax income = $13,000)

In [ ]:
situation_sgl_low = {
    "people": {
        "you": {
            "age": {"2024": "38"},
            "charitable_cash_donations": {"2024": "0"},
            "employment_income": {"2024": "13000"},
            "medical_out_of_pocket_expenses": {"2024": "200"},
        },
    },
    "families": {
        "your family": {
            "members": [
                "you",
            ]
        }
    },
    "marital_units": {
        "your marital unit": {
            "members": ["you"],
            "marital_unit_id": {"2024": 0}
        },
    },
    "tax_units": {
        "your tax unit": {
            "members": [
                "you",
            ]
        }
    },
    "spm_units": {
        "your household": {
            "members": [
                "you",
            ],
            "broadband_cost": {"2024": "400"},
            "childcare_expenses": {"2024": "0"},
            "housing_cost": {"2024": "5000"},
            "phone_cost": {"2024": "500"},
            "spm_unit_id": {"2024": "0"},
        }
    },
    "households": {
        "your household": {
            "members": [
                "you",
            ],
            "state_living_arrangement": {"2024": "FULL_COST"},
            "state_name": {"2024": "MT"}
        }
    }
}

Single no kids, middle income (household before-tax income = $40,000)


In [ ]:
situation_sgl_mid = {
    "people": {
        "you": {
            "age": {"2024": "38"},
            "charitable_cash_donations": {"2024": "2000"},
            "employment_income": {"2024": "40000"},
            "medical_out_of_pocket_expenses": {"2024": "1000"},
        },
    },
    "families": {
        "your family": {
            "members": [
                "you",
            ]
        }
    },
    "marital_units": {
        "your marital unit": {
            "members": ["you"],
            "marital_unit_id": {"2024": 0}
        },
    },
    "tax_units": {
        "your tax unit": {
            "members": [
                "you",
            ]
        }
    },
    "spm_units": {
        "your household": {
            "members": [
                "you",
            ],
            "broadband_cost": {"2024": "700"},
            "childcare_expenses": {"2024": "0"},
            "housing_cost": {"2024": "15000"},
            "phone_cost": {"2024": "1200"},
            "spm_unit_id": {"2024": "0"},
        }
    },
    "households": {
        "your household": {
            "members": [
                "you",
            ],
            "state_living_arrangement": {"2024": "FULL_COST"},
            "state_name": {"2024": "MT"}
        }
    }
}

Single, no kids, high income (household before-tax income = $90,000)

In [ ]:
situation_sgl_high = {
    "people": {
        "you": {
            "age": {"2024": "38"},
            "charitable_cash_donations": {"2024": "8000"},
            "employment_income": {"2024": "90000"},
            "medical_out_of_pocket_expenses": {"2024": "2000"},
        },
    },
    "families": {
        "your family": {
            "members": [
                "you",
            ]
        }
    },
    "marital_units": {
        "your marital unit": {
            "members": ["you"],
            "marital_unit_id": {"2024": 0}
        },
    },
    "tax_units": {
        "your tax unit": {
            "members": [
                "you",
            ]
        }
    },
    "spm_units": {
        "your household": {
            "members": [
                "you",
            ],
            "broadband_cost": {"2024": "1200"},
            "childcare_expenses": {"2024": "0"},
            "housing_cost": {"2024": "30000"},
            "phone_cost": {"2024": "2000"},
            "spm_unit_id": {"2024": "0"},
        }
    },
    "households": {
        "your household": {
            "members": [
                "you",
            ],
            "state_living_arrangement": {"2024": "FULL_COST"},
            "state_name": {"2024": "MT"}
        }
    }
}

#### 3.1.2. Define the three head of household filer types
Head of household, 2 kids, low income (household before-tax income = $18,000)

In [ ]:
situation_hoh_low = {
    "people": {
        "you": {
            "age": {"2024": "38"},
            "charitable_cash_donations": {"2024": "0"},
            "employment_income": {"2024": "20000"},
            "medical_out_of_pocket_expenses": {"2024": "500"},
        },
        "your first dependent": {
            "age": {"2024": "10"},
            "is_tax_unit_dependent": {"2024": True},
        },
        "your second dependent": {
            "age": {"2024": "6"},
            "is_tax_unit_dependent": {"2024": True},
        }
    },
    "families": {
        "your family": {
            "members": [
                "you",
                "your first dependent",
                "your second dependent"
            ]
        }
    },
    "marital_units": {
        "your marital unit": {
            "members": ["you"],
            "marital_unit_id": {"2024": 0}
        },
        "your first dependent's marital unit": {
            "members": ["your first dependent"],
            "marital_unit_id": {"2024": 2}
        },
        "your second dependent's marital unit": {
            "members": ["your second dependent"],
            "marital_unit_id": {"2024": 3}
        }
    },
    "tax_units": {
        "your tax unit": {
            "members": [
                "you",
                "your first dependent",
                "your second dependent"
            ]
        }
    },
    "spm_units": {
        "your household": {
            "members": [
                "you",
                "your first dependent",
                "your second dependent"
            ],
            "broadband_cost": {"2024": "500"},
            "childcare_expenses": {"2024": "1000"},
            "housing_cost": {"2024": "9000"},
            "phone_cost": {"2024": "600"},
            "spm_unit_id": {"2024": 0},
        }
    },
    "households": {
        "your household": {
            "members": [
                "you",
                "your first dependent",
                "your second dependent"
            ],
            "state_living_arrangement": {"2024": "FULL_COST"},
            "state_name": {"2024": "MT"}
        }
    }
}

Head of household, 2 kids, middle income (household before-tax income = $60,000)

In [ ]:
situation_hoh_mid = {
    "people": {
        "you": {
            "age": {"2024": "38"},
            "charitable_cash_donations": {"2024": "2000"},
            "employment_income": {"2024": "60000"},
            "medical_out_of_pocket_expenses": {"2024": "2000"},
        },
        "your first dependent": {
            "age": {"2024": "10"},
            "is_tax_unit_dependent": {"2024": True},
        },
        "your second dependent": {
            "age": {"2024": "6"},
            "is_tax_unit_dependent": {"2024": True},
        }
    },
    "families": {
        "your family": {
            "members": [
                "you",
                "your first dependent",
                "your second dependent"
            ]
        }
    },
    "marital_units": {
        "your marital unit": {
            "members": ["you"],
            "marital_unit_id": {"2024": 0}
        },
        "your first dependent's marital unit": {
            "members": ["your first dependent"],
            "marital_unit_id": {"2024": 2}
        },
        "your second dependent's marital unit": {
            "members": ["your second dependent"],
            "marital_unit_id": {"2024": 3}
        }
    },
    "tax_units": {
        "your tax unit": {
            "members": [
                "you",
                "your first dependent",
                "your second dependent"
            ]
        }
    },
    "spm_units": {
        "your household": {
            "members": [
                "you",
                "your first dependent",
                "your second dependent"
            ],
            "broadband_cost": {"2024": "800"},
            "childcare_expenses": {"2024": "2000"},
            "housing_cost": {"2024": "20000"},
            "phone_cost": {"2024": "1000"},
            "spm_unit_id": {"2024": 0},
        }
    },
    "households": {
        "your household": {
            "members": [
                "you",
                "your first dependent",
                "your second dependent"
            ],
            "state_living_arrangement": {"2024": "FULL_COST"},
            "state_name": {"2024": "MT"}
        }
    }
}

Head of household, 2 kids, high income (household before-tax income = $150,000)

In [ ]:
situation_hoh_high = {
    "people": {
        "you": {
            "age": {"2024": "38"},
            "charitable_cash_donations": {"2024": "15000"},
            "employment_income": {"2024": "150000"},
            "medical_out_of_pocket_expenses": {"2024": "3000"},
        },
        "your first dependent": {
            "age": {"2024": "10"},
            "is_tax_unit_dependent": {"2024": True},
        },
        "your second dependent": {
            "age": {"2024": "6"},
            "is_tax_unit_dependent": {"2024": True},
        }
    },
    "families": {
        "your family": {
            "members": [
                "you",
                "your first dependent",
                "your second dependent"
            ]
        }
    },
    "marital_units": {
        "your marital unit": {
            "members": ["you"],
            "marital_unit_id": {"2024": 0}
        },
        "your first dependent's marital unit": {
            "members": ["your first dependent"],
            "marital_unit_id": {"2024": 2}
        },
        "your second dependent's marital unit": {
            "members": ["your second dependent"],
            "marital_unit_id": {"2024": 3}
        }
    },
    "tax_units": {
        "your tax unit": {
            "members": [
                "you",
                "your first dependent",
                "your second dependent"
            ]
        }
    },
    "spm_units": {
        "your household": {
            "members": [
                "you",
                "your first dependent",
                "your second dependent"
            ],
            "broadband_cost": {"2024": "1200"},
            "childcare_expenses": {"2024": "3000"},
            "housing_cost": {"2024": "36000"},
            "phone_cost": {"2024": "1800"},
            "spm_unit_id": {"2024": 0},
        }
    },
    "households": {
        "your household": {
            "members": [
                "you",
                "your first dependent",
                "your second dependent"
            ],
            "state_living_arrangement": {"2024": "FULL_COST"},
            "state_name": {"2024": "MT"}
        }
    }
}

#### 3.1.3. Define the six married filing jointly filer types
Married filing jointly, 0 kids, low income (household before-tax income = $22,000)

In [ ]:
situation_mar_0_low = {
    "people": {
        "you": {
            "age": {"2024": "38"},
            "charitable_cash_donations": {"2024": "0"},
            "employment_income": {"2024": "12000"},
            "medical_out_of_pocket_expenses": {"2024": "400"},
        },
        "your partner": {
            "age": {"2024": "35"},
            "employment_income": {"2024": "10000"},
        }
    },
    "families": {
        "your family": {
            "members": [
                "you",
                "your partner"
            ]
        }
    },
    "marital_units": {
        "your marital unit": {
            "members": ["you", "your partner"],
            "marital_unit_id": {"2024": 0}
        },
    },
    "tax_units": {
        "your tax unit": {
            "members": [
                "you",
                "your partner"
            ]
        }
    },
    "spm_units": {
        "your household": {
            "members": [
                "you",
                "your partner"
            ],
            "broadband_cost": {"2024": "500"},
            "childcare_expenses": {"2024": "0"},
            "housing_cost": {"2024": "6000"},
            "phone_cost": {"2024": "800"},
            "spm_unit_id": {"2024": 0},
        }
    },
    "households": {
        "your household": {
            "members": [
                "you",
                "your partner"
            ],
            "state_living_arrangement": {"2024": "FULL_COST"},
            "state_name": {"2024": "MT"}
        }
    }
}

Married filing jointly, 0 kids, middle income (household before-tax income = $60,000)

In [ ]:
situation_mar_0_mid = {
    "people": {
        "you": {
            "age": {"2024": "38"},
            "charitable_cash_donations": {"2024": "2000"},
            "employment_income": {"2024": "35000"},
            "medical_out_of_pocket_expenses": {"2024": "1800"},
        },
        "your partner": {
            "age": {"2024": "35"},
            "employment_income": {"2024": "25000"},
        }
    },
    "families": {
        "your family": {
            "members": [
                "you",
                "your partner"
            ]
        }
    },
    "marital_units": {
        "your marital unit": {
            "members": ["you", "your partner"],
            "marital_unit_id": {"2024": 0}
        },
    },
    "tax_units": {
        "your tax unit": {
            "members": [
                "you",
                "your partner"
            ]
        }
    },
    "spm_units": {
        "your household": {
            "members": [
                "you",
                "your partner"
            ],
            "broadband_cost": {"2024": "900"},
            "childcare_expenses": {"2024": "0"},
            "housing_cost": {"2024": "18000"},
            "phone_cost": {"2024": "1200"},
            "spm_unit_id": {"2024": 0},
        }
    },
    "households": {
        "your household": {
            "members": [
                "you",
                "your partner"
            ],
            "state_living_arrangement": {"2024": "FULL_COST"},
            "state_name": {"2024": "MT"}
        }
    }
}

Married filing jointly, 0 kids, high income (household before-tax income = $160,000)

In [ ]:
situation_mar_0_high = {
    "people": {
        "you": {
            "age": {"2024": "38"},
            "charitable_cash_donations": {"2024": "16000"},
            "employment_income": {"2024": "90000"},
            "medical_out_of_pocket_expenses": {"2024": "3000"},
        },
        "your partner": {
            "age": {"2024": "35"},
            "employment_income": {"2024": "70000"},
        }
    },
    "families": {
        "your family": {
            "members": [
                "you",
                "your partner"
            ]
        }
    },
    "marital_units": {
        "your marital unit": {
            "members": ["you", "your partner"],
            "marital_unit_id": {"2024": 0}
        },
    },
    "tax_units": {
        "your tax unit": {
            "members": [
                "you",
                "your partner"
            ]
        }
    },
    "spm_units": {
        "your household": {
            "members": [
                "you",
                "your partner"
            ],
            "broadband_cost": {"2024": "1200"},
            "childcare_expenses": {"2024": "0"},
            "housing_cost": {"2024": "36000"},
            "phone_cost": {"2024": "2000"},
            "spm_unit_id": {"2024": 0},
        }
    },
    "households": {
        "your household": {
            "members": [
                "you",
                "your partner"
            ],
            "state_living_arrangement": {"2024": "FULL_COST"},
            "state_name": {"2024": "MT"}
        }
    }
}

Married filing jointly, 2 kids, low income (household before-tax income = $25,000)

In [ ]:
situation_mar_2_low = {
    "people": {
        "you": {
            "age": {"2024": "38"},
            "charitable_cash_donations": {"2024": "0"},
            "employment_income": {"2024": "25000"},
            "medical_out_of_pocket_expenses": {"2024": "600"},
        },
        "your partner": {
            "age": {"2024": "35"},
            "employment_income": {"2024": "0"},
        },
        "your first dependent": {
            "age": {"2024": "10"},
            "is_tax_unit_dependent": {"2024": True},
        },
        "your second dependent": {
            "age": {"2024": "6"},
            "is_tax_unit_dependent": {"2024": True},
        }
    },
    "families": {
        "your family": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ]
        }
    },
    "marital_units": {
        "your marital unit": {
            "members": ["you", "your partner"],
            "marital_unit_id": {"2024": 0}
        },
        "your first dependent's marital unit": {
            "members": ["your first dependent"],
            "marital_unit_id": {"2024": 2}
        },
        "your second dependent's marital unit": {
            "members": ["your second dependent"],
            "marital_unit_id": {"2024": 3}
        }
    },
    "tax_units": {
        "your tax unit": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ]
        }
    },
    "spm_units": {
        "your household": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ],
            "broadband_cost": {"2024": "500"},
            "childcare_expenses": {"2024": "1000"},
            "housing_cost": {"2024": "10000"},
            "phone_cost": {"2024": "800"},
            "spm_unit_id": {"2024": 0},
        }
    },
    "households": {
        "your household": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ],
            "state_living_arrangement": {"2024": "FULL_COST"},
            "state_name": {"2024": "MT"}
        }
    }
}

Married filing jointly, 2 kids, middle income (household before-tax income = $70,000)

In [ ]:
situation_mar_2_mid = {
    "people": {
        "you": {
            "age": {"2024": "38"},
            "charitable_cash_donations": {"2024": "2500"},
            "employment_income": {"2024": "40000"},
            "medical_out_of_pocket_expenses": {"2024": "2500"},
        },
        "your partner": {
            "age": {"2024": "35"},
            "employment_income": {"2024": "30000"},
        },
        "your first dependent": {
            "age": {"2024": "10"},
            "is_tax_unit_dependent": {"2024": True},
        },
        "your second dependent": {
            "age": {"2024": "6"},
            "is_tax_unit_dependent": {"2024": True},
        }
    },
    "families": {
        "your family": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ]
        }
    },
    "marital_units": {
        "your marital unit": {
            "members": ["you", "your partner"],
            "marital_unit_id": {"2024": 0}
        },
        "your first dependent's marital unit": {
            "members": ["your first dependent"],
            "marital_unit_id": {"2024": 2}
        },
        "your second dependent's marital unit": {
            "members": ["your second dependent"],
            "marital_unit_id": {"2024": 3}
        }
    },
    "tax_units": {
        "your tax unit": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ]
        }
    },
    "spm_units": {
        "your household": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ],
            "broadband_cost": {"2024": "900"},
            "childcare_expenses": {"2024": "2200"},
            "housing_cost": {"2024": "22000"},
            "phone_cost": {"2024": "1500"},
            "spm_unit_id": {"2024": 0},
        }
    },
    "households": {
        "your household": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ],
            "state_living_arrangement": {"2024": "FULL_COST"},
            "state_name": {"2024": "MT"}
        }
    }
}

Married filing jointly, 2 kids, high income (household before-tax income = $200,000)

In [ ]:
situation_mar_2_high = {
    "people": {
        "you": {
            "age": {"2024": "38"},
            "charitable_cash_donations": {"2024": "20000"},
            "employment_income": {"2024": "100000"},
            "medical_out_of_pocket_expenses": {"2024": "3500"},
        },
        "your partner": {
            "age": {"2024": "35"},
            "employment_income": {"2024": "100000"},
        },
        "your first dependent": {
            "age": {"2024": "10"},
            "is_tax_unit_dependent": {"2024": True},
        },
        "your second dependent": {
            "age": {"2024": "6"},
            "is_tax_unit_dependent": {"2024": True},
        }
    },
    "families": {
        "your family": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ]
        }
    },
    "marital_units": {
        "your marital unit": {
            "members": ["you", "your partner"],
            "marital_unit_id": {"2024": 0}
        },
        "your first dependent's marital unit": {
            "members": ["your first dependent"],
            "marital_unit_id": {"2024": 2}
        },
        "your second dependent's marital unit": {
            "members": ["your second dependent"],
            "marital_unit_id": {"2024": 3}
        }
    },
    "tax_units": {
        "your tax unit": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ]
        }
    },
    "spm_units": {
        "your household": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ],
            "broadband_cost": {"2024": "1200"},
            "childcare_expenses": {"2024": "3000"},
            "housing_cost": {"2024": "36000"},
            "phone_cost": {"2024": "1800"},
            "spm_unit_id": {"2024": 0},
        }
    },
    "households": {
        "your household": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ],
            "state_living_arrangement": {"2024": "FULL_COST"},
            "state_name": {"2024": "MT"}
        }
    }
}

### 3.2. Run all baseline and reform simulations

In [ ]:
# Create baseline (no change) parameters
def modify_parameters_b(parameters):
    """
    Baseline reform is to not modify the parameters.
    """
    pass
    return parameters


# Create  baseline (no change) reform
class reform_b(Reform):
    def apply(self):
        self.modify_parameters(modify_parameters_b)


# Define the parameters that change in the reform
def modify_parameters_1(parameters):
    # Defining flat tax
    parameters.gov.states.mt.tax.income.rates.joint[0].rate.update(
        start=instant("2024-01-01"), stop=instant("2028-12-31"), value=0.052
    )
    parameters.gov.states.mt.tax.income.rates.joint[1].rate.update(
        start=instant("2024-01-01"), stop=instant("2028-12-31"), value=0.052
    )
    parameters.gov.states.mt.tax.income.rates.head_of_household[0].rate.update(
        start=instant("2024-01-01"), stop=instant("2028-12-31"), value=0.052
    )
    parameters.gov.states.mt.tax.income.rates.head_of_household[1].rate.update(
        start=instant("2024-01-01"), stop=instant("2028-12-31"), value=0.052
    )
    parameters.gov.states.mt.tax.income.rates.separate[0].rate.update(
        start=instant("2024-01-01"), stop=instant("2028-12-31"), value=0.052
    )
    parameters.gov.states.mt.tax.income.rates.separate[1].rate.update(
        start=instant("2024-01-01"), stop=instant("2028-12-31"), value=0.052
    )
    parameters.gov.states.mt.tax.income.rates.single[0].rate.update(
        start=instant("2024-01-01"), stop=instant("2028-12-31"), value=0.052
    )
    parameters.gov.states.mt.tax.income.rates.single[1].rate.update(
        start=instant("2024-01-01"), stop=instant("2028-12-31"), value=0.052
    )
    # Increasing the standard deduction rate
    parameters.gov.states.mt.tax.income.deductions.standard.rate.update(
        start=instant("2024-01-01"), stop=instant("2028-12-31"), value=0.27
    )
    # Increasing the standard deduction minimum
    parameters.gov.states.mt.tax.income.deductions.standard.min.JOINT.update(
        start=instant("2024-01-01"), stop=instant("2028-12-31"), value=5_420
    )
    parameters.gov.states.mt.tax.income.deductions.standard.min.HEAD_OF_HOUSEHOLD.update(
        start=instant("2024-01-01"), stop=instant("2028-12-31"), value=5_020
    )
    parameters.gov.states.mt.tax.income.deductions.standard.min.WIDOW.update(
        start=instant("2024-01-01"), stop=instant("2028-12-31"), value=2_760
    )
    parameters.gov.states.mt.tax.income.deductions.standard.min.SINGLE.update(
        start=instant("2024-01-01"), stop=instant("2028-12-31"), value=2_760
    )
    parameters.gov.states.mt.tax.income.deductions.standard.min.SEPARATE.update(
        start=instant("2024-01-01"), stop=instant("2028-12-31"), value=2_760
    )
    # Increasing the standard deduction max
    parameters.gov.states.mt.tax.income.deductions.standard.max.JOINT.update(
        start=instant("2024-01-01"), stop=instant("2028-12-31"), value=14_180
    )
    parameters.gov.states.mt.tax.income.deductions.standard.max.HEAD_OF_HOUSEHOLD.update(
        start=instant("2024-01-01"), stop=instant("2028-12-31"), value=14_180
    )
    parameters.gov.states.mt.tax.income.deductions.standard.max.WIDOW.update(
        start=instant("2024-01-01"), stop=instant("2028-12-31"), value=7_090
    )
    parameters.gov.states.mt.tax.income.deductions.standard.max.SINGLE.update(
        start=instant("2024-01-01"), stop=instant("2028-12-31"), value=7_090
    )
    parameters.gov.states.mt.tax.income.deductions.standard.max.SEPARATE.update(
        start=instant("2024-01-01"), stop=instant("2028-12-31"), value=7_090
    )

    return parameters


class reform_1(Reform):
    def apply(self):
        self.modify_parameters(modify_parameters_1)


def calculate_base_reform(sim_name, base_reform, reform, situation):
    print("Simulating baseline scenario for:", sim_name)
    simulation_b = Simulation(reform=base_reform, situation=situation)
    # simulation_b.trace = True
    income_before_tax = sum(simulation_b.calculate("employment_income", 2024))
    income_after_tax_b = simulation_b.calculate("household_net_income", 2024)[0]
    mt_net_tax_liability_b = simulation_b.calculate("mt_income_tax", 2024)[0]
    print("Simulating reform scenario for:", sim_name)
    simulation_r = Simulation(reform=reform, situation=situation)
    # simulation_r.trace = True
    income_after_tax_r = simulation_r.calculate("household_net_income", 2024)[0]
    mt_net_tax_liability_r = simulation_r.calculate("mt_income_tax", 2024)[0]
    mt_net_tax_liab_dol_chg = mt_net_tax_liability_r - mt_net_tax_liability_b
    mt_net_tax_liab_pct_chg = (mt_net_tax_liab_dol_chg /
                               np.absolute(mt_net_tax_liability_b))
    print("")
    return (
        income_before_tax, income_after_tax_b, mt_net_tax_liability_b,
        income_after_tax_r, mt_net_tax_liability_r, mt_net_tax_liab_dol_chg,
        mt_net_tax_liab_pct_chg
    )

simulations = [
    ('Single, no kids, low income', situation_sgl_low),
    ('Single, no kids, middle income', situation_sgl_mid),
    ('Single, no kids, high income', situation_sgl_high),
    ('Married filing jointly, no kids, low income', situation_mar_0_low),
    ('Married filing jointly, no kids, middle income', situation_mar_0_mid),
    ('Married filing jointly, no kids, high income', situation_mar_0_high),
    ('Head of household, 2 kids, low income', situation_hoh_low),
    ('Head of household, 2 kids, middle income', situation_hoh_mid),
    ('Head of household, 2 kids, high income', situation_hoh_high),
    ('Married, 2 kids, low income', situation_mar_2_low),
    ('Married, 2 kids, middle income', situation_mar_2_mid),
    ('Married, 2 kids, high income', situation_mar_2_high),
]

results = []

# Loop through each simulation
for sim_name, situation in simulations:
    # Calculate before_tax_income, after_tax_income, and mt_net_tax_liability
    (income_before_tax, income_after_tax_b, mt_net_tax_liability_b,
     income_after_tax_r, mt_net_tax_liability_r, mt_net_tax_liab_dol_chg,
     mt_net_tax_liab_pct_chg) = calculate_base_reform(sim_name, reform_b,
                                                      reform_1, situation)

    # Append the results as a dictionary where key is the column name and value
    # is the simulation output
    results.append({
        "Situation": sim_name,
        "Before tax income": income_before_tax,
        "Baseline after tax income": income_after_tax_b,
        "Baseline Montana net tax liability": mt_net_tax_liability_b,
        "Reform after tax income": income_after_tax_r,
        "Reform Montana net tax liability": mt_net_tax_liability_r,
        "Montana net tax liability change, dollars": mt_net_tax_liab_dol_chg,
        "Montana net tax liability change, percent": mt_net_tax_liab_pct_chg
    })

# Convert the results to a DataFrame
df = pd.DataFrame(results)

# Check the DataFrame
df

### 3.3. Make plot of Montana net tax liability by before-tax income for 4 filer types

#### 3.3.1. Create the four new filers with employment income allowed to vary

In [ ]:
inc_min = 0
inc_max = 200_000
num_points = 10001

Create new Single filer with two children, the same other characteristics as the middle income type, and income varying.

In [ ]:
situation_sgl_mid_plot = {
    "people": {
        "you": {
            "age": {"2024": "38"},
            "charitable_cash_donations": {"2024": "2000"},
            "medical_out_of_pocket_expenses": {"2024": "1000"},
        },
    },
    "families": {
        "your family": {
            "members": [
                "you",
            ]
        }
    },
    "marital_units": {
        "your marital unit": {
            "members": ["you"],
            "marital_unit_id": {"2024": 0}
        },
    },
    "tax_units": {
        "your tax unit": {
            "members": [
                "you",
            ]
        }
    },
    "spm_units": {
        "your household": {
            "members": [
                "you",
            ],
            "broadband_cost": {"2024": "700"},
            "childcare_expenses": {"2024": "0"},
            "housing_cost": {"2024": "15000"},
            "phone_cost": {"2024": "1200"},
            "spm_unit_id": {"2024": "0"},
        }
    },
    "households": {
        "your household": {
            "members": [
                "you",
            ],
            "state_living_arrangement": {"2024": "FULL_COST"},
            "state_name": {"2024": "MT"}
        }
    },
    "axes": [
        [
            {
                "name": "employment_income",
                "count": num_points,
                "min": inc_min,
                "max": inc_max
            }
        ]
    ]
}


Create new Married filing jointly filer with no children, the same other characteristics as the middle income type, and income varying.

In [ ]:
situation_mar_0_mid_plot = {
    "people": {
        "you": {
            "age": {"2024": "38"},
            "charitable_cash_donations": {"2024": "2000"},
            "medical_out_of_pocket_expenses": {"2024": "1800"},
        },
        "your partner": {
            "age": {"2024": "35"},
        }
    },
    "families": {
        "your family": {
            "members": [
                "you",
                "your partner"
            ]
        }
    },
    "marital_units": {
        "your marital unit": {
            "members": ["you", "your partner"],
            "marital_unit_id": {"2024": 0}
        },
    },
    "tax_units": {
        "your tax unit": {
            "members": [
                "you",
                "your partner"
            ]
        }
    },
    "spm_units": {
        "your household": {
            "members": [
                "you",
                "your partner"
            ],
            "broadband_cost": {"2024": "900"},
            "childcare_expenses": {"2024": "0"},
            "housing_cost": {"2024": "18000"},
            "phone_cost": {"2024": "1200"},
            "spm_unit_id": {"2024": 0},
        }
    },
    "households": {
        "your household": {
            "members": [
                "you",
                "your partner"
            ],
            "state_living_arrangement": {"2024": "FULL_COST"},
            "state_name": {"2024": "MT"}
        }
    },
    "axes": [
        [
            {
                "name": "employment_income",
                "count": num_points,
                "min": inc_min,
                "max": inc_max
            }
        ]
    ]
}


Create new Head of household filer with two children, the same other characteristics as the middle income type, and income varying.

In [ ]:
situation_hoh_mid_plot = {
    "people": {
        "you": {
            "age": {"2024": "38"},
            "charitable_cash_donations": {"2024": "2000"},
            "medical_out_of_pocket_expenses": {"2024": "2000"},
        },
        "your first dependent": {
            "age": {"2024": "10"},
            "is_tax_unit_dependent": {"2024": True},
        },
        "your second dependent": {
            "age": {"2024": "6"},
            "is_tax_unit_dependent": {"2024": True},
        }
    },
    "families": {
        "your family": {
            "members": [
                "you",
                "your first dependent",
                "your second dependent"
            ]
        }
    },
    "marital_units": {
        "your marital unit": {
            "members": ["you"],
            "marital_unit_id": {"2024": 0}
        },
        "your first dependent's marital unit": {
            "members": ["your first dependent"],
            "marital_unit_id": {"2024": 2}
        },
        "your second dependent's marital unit": {
            "members": ["your second dependent"],
            "marital_unit_id": {"2024": 3}
        }
    },
    "tax_units": {
        "your tax unit": {
            "members": [
                "you",
                "your first dependent",
                "your second dependent"
            ]
        }
    },
    "spm_units": {
        "your household": {
            "members": [
                "you",
                "your first dependent",
                "your second dependent"
            ],
            "broadband_cost": {"2024": "800"},
            "childcare_expenses": {"2024": "2000"},
            "housing_cost": {"2024": "20000"},
            "phone_cost": {"2024": "1000"},
            "spm_unit_id": {"2024": 0},
        }
    },
    "households": {
        "your household": {
            "members": [
                "you",
                "your first dependent",
                "your second dependent"
            ],
            "state_living_arrangement": {"2024": "FULL_COST"},
            "state_name": {"2024": "MT"}
        }
    },
    "axes": [
        [
            {
                "name": "employment_income",
                "count": num_points,
                "min": inc_min,
                "max": inc_max
            }
        ]
    ]
}


Create new Married filing jointly filer with two children, the same other characteristics as the middle income type, and income varying.

In [ ]:
situation_mar_2_mid_plot = {
    "people": {
        "you": {
            "age": {"2024": "38"},
            "charitable_cash_donations": {"2024": "2500"},
            "medical_out_of_pocket_expenses": {"2024": "2500"},
        },
        "your partner": {
            "age": {"2024": "35"},
        },
        "your first dependent": {
            "age": {"2024": "10"},
            "is_tax_unit_dependent": {"2024": True},
        },
        "your second dependent": {
            "age": {"2024": "6"},
            "is_tax_unit_dependent": {"2024": True},
        }
    },
    "families": {
        "your family": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ]
        }
    },
    "marital_units": {
        "your marital unit": {
            "members": ["you", "your partner"],
            "marital_unit_id": {"2024": 0}
        },
        "your first dependent's marital unit": {
            "members": ["your first dependent"],
            "marital_unit_id": {"2024": 2}
        },
        "your second dependent's marital unit": {
            "members": ["your second dependent"],
            "marital_unit_id": {"2024": 3}
        }
    },
    "tax_units": {
        "your tax unit": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ]
        }
    },
    "spm_units": {
        "your household": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ],
            "broadband_cost": {"2024": "900"},
            "childcare_expenses": {"2024": "2200"},
            "housing_cost": {"2024": "22000"},
            "phone_cost": {"2024": "1500"},
            "spm_unit_id": {"2024": 0},
        }
    },
    "households": {
        "your household": {
            "members": [
                "you",
                "your partner",
                "your first dependent",
                "your second dependent"
            ],
            "state_living_arrangement": {"2024": "FULL_COST"},
            "state_name": {"2024": "MT"}
        }
    },
    "axes": [
        [
            {
                "name": "employment_income",
                "count": num_points,
                "min": inc_min,
                "max": inc_max
            }
        ]
    ]
}


#### 3.3.2. Create the plot of the four filer types varying by income

Create the data series for the four lines of the plot.

In [ ]:
def calculate_base_reform_plot(sim_name, base_reform, reform, situation):
    print("Simulating baseline scenario for plot:", sim_name)
    simulation_b = Simulation(reform=base_reform, situation=situation)
    simulation_b.trace = True
    income_before_tax_vec = simulation_b.calculate("employment_income", 2024)
    income_after_tax_b_vec = simulation_b.calculate("household_net_income",
                                                    2024)
    mt_net_tax_liability_b_vec = simulation_b.calculate("mt_income_tax", 2024)
    print("Simulating reform scenario for plot:", sim_name)
    simulation_r = Simulation(reform=reform, situation=situation)
    simulation_r.trace = True
    income_after_tax_r_vec = simulation_r.calculate("household_net_income",
                                                    2024)
    mt_net_tax_liability_r_vec = simulation_r.calculate("mt_income_tax", 2024)
    mt_net_tax_liab_dol_chg_vec = (mt_net_tax_liability_r_vec -
                                   mt_net_tax_liability_b_vec)
    mt_net_tax_liab_pct_chg_vec = (mt_net_tax_liab_dol_chg_vec /
                                   np.absolute(mt_net_tax_liability_b_vec))
    print("")
    return (
        income_before_tax_vec, income_after_tax_b_vec,
        mt_net_tax_liability_b_vec, income_after_tax_r_vec,
        mt_net_tax_liability_r_vec, mt_net_tax_liab_dol_chg_vec,
        mt_net_tax_liab_pct_chg_vec
    )

results_plot = []
df_list = []
cds_list = []
filer_type_label_list = []

sims_to_plot = [
    ('Single, no kids', situation_sgl_mid_plot),
    ('Married filing jointly, no kids', situation_mar_0_mid_plot),
    ('Head of household, 2 kids', situation_hoh_mid_plot),
    ('Married filing jointly, 2 kids', situation_mar_2_mid_plot)
]

# Loop through each simulation with varying income
for sim_name, situation in sims_to_plot:
    # Calculate before_tax_income, after_tax_income, and ks_net_tax_liability
    (inc_before_tax_vec, inc_after_tax_b_vec, mt_net_tax_liab_b_vec,
     inc_after_tax_r_vec, mt_net_tax_liab_r_vec,
     mt_net_tax_liab_dol_chg_vec, mt_net_tax_liab_pct_chg_vec) = \
        calculate_base_reform_plot(sim_name, reform_b, reform_1, situation)
    if sim_name == "Head of household, 2 kids":
        inc_before_tax_vec_weird = inc_before_tax_vec.copy()

    # Create a DataFrame
    data_df = pd.DataFrame([])
    if sim_name == "Single, no kids":
        data_df["inc_before_tax"] = inc_before_tax_vec
    elif sim_name == "Married filing jointly, no kids":
        # Income before tax array gives incomes for both spouses for Married
        # filing jointly, no kids. Take only even indices 0, 2, 4,... .
        data_df["inc_before_tax"] = inc_before_tax_vec[0::2]
    elif sim_name == "Head of household, 2 kids":
        # Income before tax array gives incomes for head of household and for
        # both children for Head of household, 2 kids. Take only every third
        # index 0, 3, 6,... .
        data_df["inc_before_tax"] = inc_before_tax_vec[0::3]
    elif sim_name == "Married filing jointly, 2 kids":
        # Income before tax array gives incomes for both spouses and for both
        # children for Married filing jointly, 2 kids. Take only every fourth
        # index 0, 4, 8,... .
        data_df["inc_before_tax"] = inc_before_tax_vec[0::4]
    data_df["inc_after_tax_b"] = inc_after_tax_b_vec
    data_df["mt_net_tax_liab_b"] = mt_net_tax_liab_b_vec
    data_df["inc_after_tax_r"] = inc_after_tax_r_vec
    data_df["mt_net_tax_liab_r"] = mt_net_tax_liab_r_vec
    data_df["mt_net_tax_liab_dol_chg"] = mt_net_tax_liab_dol_chg_vec
    data_df["mt_net_tax_liab_pct_chg"] = mt_net_tax_liab_pct_chg_vec
    data_df["filer_type"] = sim_name

    # Append the sim name to the filer_type_label_list
    filer_type_label_list.append(sim_name)

    # Append the results as a dictionary where key is the column name and value
    # is the simulation output
    results_plot.append({
        "Situation": sim_name,
        "Before tax income vector": inc_before_tax_vec,
        "Baseline Montana net tax liability vector": mt_net_tax_liab_b_vec,
        "Reform Montana net tax liability vector": mt_net_tax_liab_r_vec,
        "Montana net tax liability change vector, dollars": mt_net_tax_liab_dol_chg_vec,
        "Pandas DataFrame": data_df
    })

    # Append the data_df as a Pandas DataFrame object to the df_list
    df_list.append(data_df)

    # Append the data_df as a ColumnDataSource object to the cds_list
    cds_list.append(ColumnDataSource(data_df))


In [ ]:
# Save source data for figure 4
fig4_source_df = df_list[0][["inc_before_tax", "mt_net_tax_liab_dol_chg"]].rename(columns={"mt_net_tax_liab_dol_chg": "mt_net_tax_liab_dol_chg_single"})
fig4_source_df["mt_net_tax_liab_dol_chg_mar0kids"] = df_list[1]["mt_net_tax_liab_dol_chg"]
fig4_source_df["mt_net_tax_liab_dol_chg_hoh2kids"] = df_list[2]["mt_net_tax_liab_dol_chg"]
fig4_source_df["mt_net_tax_liab_dol_chg_mar2kids"] = df_list[3]["mt_net_tax_liab_dol_chg"]
fig4_source_df.to_csv('./data/fig4_source.csv', index=False)

Create the Bokeh plot for Figure 4.

In [ ]:
# Create Bokeh plot of Figure 4 change in Montana net tax liability
fig4_title = (
    "Fig. 4. Dollar change from MT flat tax reform in filer net state " +
    "income tax liability"
)
# fig4_title = ""
filename4 = "./images/NetStateTaxLiabChg.html"
output_file(filename4, title=fig4_title, mode='inline')
output_notebook()

# Format the tooltip
tooltips = [
    ("Filer type", "@filer_type"),
    ("Before-tax employment income", "$x{$0,0.}"),
    ("Change in Montana net tax liability", "$y{$0,0.}"),
]

# Solve for minimum and maximum before-tax income and change in tax liability
# values in order to set the appropriate xrange and yrange
min_xval = inc_min
max_xval = inc_max
datarange_xvals = inc_max - inc_min
min_yval = 1e10
max_yval = -1e10
for filer_type in results_plot:
    min_yval = np.minimum(
        min_yval, filer_type[
            "Montana net tax liability change vector, dollars"
        ].min()
    )
    max_yval = np.maximum(
        max_yval, filer_type[
            "Montana net tax liability change vector, dollars"
        ].max()
    )
datarange_yvals = max_yval - min_yval
fig4_buffer_pct = 0.05
fig4 = figure(
    height=500,
    width=800,
    x_axis_label="Before-tax employment income",
    y_axis_label="Change in Montana net tax liability",
    y_range=(
        -800,  # min_yval,
        max_yval + fig4_buffer_pct * datarange_yvals
    ),
    x_range=(
        min_xval,
        150000  # max_xval
    ),
    tools=[
        "save",
        "zoom_in",
        "zoom_out",
        "box_zoom",
        "pan",
        "undo",
        "redo",
        "reset",
        "hover",
        "help",
    ],
    toolbar_location="right",
)
fig4.toolbar.logo = None

l0 = fig4.line(
    x="inc_before_tax",
    y="mt_net_tax_liab_dol_chg",
    source=cds_list[0],
    color="blue",  # Category10[4][0],
    line_width=3,
    alpha=0.7,
    muted_alpha=0.15,
)
l1 = fig4.line(
    x="inc_before_tax",
    y="mt_net_tax_liab_dol_chg",
    source=cds_list[1],
    color="purple",  # Category10[4][1],
    line_width=3,
    alpha=0.7,
    muted_alpha=0.15,
)
l2 = fig4.line(
    x="inc_before_tax",
    y="mt_net_tax_liab_dol_chg",
    source=cds_list[2],
    color="green",  # Category10[4][2],
    line_width=3,
    alpha=0.7,
    muted_alpha=0.15,
)
l3 = fig4.line(
    x="inc_before_tax",
    y="mt_net_tax_liab_dol_chg",
    source=cds_list[3],
    color="red",  # Category10[4][3],
    line_width=3,
    alpha=0.7,
    muted_alpha=0.15,
)

# Dashed horizontal line at $0 change
fig4.line(
    x=[min_xval - fig4_buffer_pct * datarange_xvals,
       max_xval + fig4_buffer_pct * datarange_xvals],
    y=[0.0, 0.0],
    color="black",
    line_width=2,
    line_dash="dashed",
    alpha=0.5,
)

# Add title
fig4.add_layout(
    Title(
        text=fig4_title,
        text_font_style="bold",
        text_font_size="14pt",
        align="center",
    ),
    "above",
)

# Add legend
legend = Legend(
    items=[
        (filer_type_label_list[0], [l0]),
        (filer_type_label_list[1], [l1]),
        (filer_type_label_list[2], [l2]),
        (filer_type_label_list[3], [l3]),
    ],
    # location="center",
)
fig4.add_layout(legend)
fig4.legend.location = "top_right"
fig4.legend.click_policy = "mute"

# Add the HoverTool to the figure
fig4.add_tools(
    HoverTool(
        tooltips=tooltips,
        toggleable=False,
    )
)

# Customize the x-axis and y-axis ticks and tick lables and gridlines
fig4.xaxis.ticker = [0, 25_000, 50_000, 75_000, 100_000, 125_000, 150_000,
                     175_000, 200_000]
fig4.xaxis.major_label_overrides = {
    0: '$0', 25_000: '$25k', 50_000: '$50k', 75_000: '$75k', 100_000: '$100k',
    125_000: '$125k', 150_000: '$150k', 175_000: '$175k', 200_000: '$200k'
}
fig4.yaxis.ticker = [
    -1200, -1000, -800, -600, -400, -200, 0
]
fig4.yaxis.major_label_overrides = {
    -1200: '-$1,200', -1000: '-$1,000', -800: '-$800', -600: '-$600',
    -400: '-$400', -200: '-$200', 0: '$0'
}

# Add source text below figure
fig4.add_layout(
    Title(
        # text="Source: Richard W. Evans (@RickEcon), FiscalSim open source " +
        # "microsimulation model of federal and state individual tax and " +
        # "benefit policy.",
        text="Source: FiscalSim-US open source " +
        "microsimulation model of federal and state individual tax and " +
        "benefit policy.",
        align="left",
        text_font_size="3mm",
        text_font_style="italic",
    ),
    "below",
)

show(fig4)
